In [1]:
import io
import numpy as np
import PIL
import requests
import openpifpaf
import torch
import cv2
import json
import time
from PIL import Image
from pymongo import MongoClient
from datetime import datetime

print('OpenPifPaf version', openpifpaf.__version__)
print('PyTorch version', torch.__version__)

OpenPifPaf version 0.13.4
PyTorch version 1.11.0


In [2]:
image_response = requests.get('https://raw.githubusercontent.com/openpifpaf/openpifpaf/main/docs/coco/000000081988.jpg')
pil_im = PIL.Image.open(io.BytesIO(image_response.content)).convert('RGB')
print(pil_im)
im = np.asarray(pil_im)
print(pil_im)

with openpifpaf.show.image_canvas(im) as ax:
    pass

<PIL.Image.Image image mode=RGB size=640x427 at 0x7F879AC25C40>
<PIL.Image.Image image mode=RGB size=640x427 at 0x7F879AC25C40>


In [3]:
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')
predictions, gt_anns, image_meta = predictor.pil_image(pil_im)

src/openpifpaf/csrc/src/cif_hr.cpp:102: UserInfo: resizing cifhr buffer
src/openpifpaf/csrc/src/occupancy.cpp:53: UserInfo: resizing occupancy buffer


In [4]:
annotation_painter = openpifpaf.show.AnnotationPainter()
with openpifpaf.show.image_canvas(im) as ax:
    annotation_painter.annotations(ax, predictions)

In [5]:
predictions[0]

In [6]:
predictions[0].data

array([[ 3.6017807e+02,  2.9964923e+02,  9.9882036e-01],
       [ 3.6403445e+02,  2.9479434e+02,  9.5610690e-01],
       [ 3.5507327e+02,  2.9503882e+02,  9.7656912e-01],
       [ 3.6957373e+02,  2.9712720e+02,  8.2809377e-01],
       [ 3.4776581e+02,  2.9803564e+02,  9.4154650e-01],
       [ 3.8178824e+02,  3.1739838e+02,  9.3544292e-01],
       [ 3.4123199e+02,  3.2180969e+02,  9.5377135e-01],
       [ 3.8761176e+02,  3.4162183e+02,  6.1720490e-01],
       [ 3.3507950e+02,  3.5091245e+02,  9.6102774e-01],
       [ 3.7334363e+02,  3.5688348e+02,  4.3549511e-01],
       [ 3.3581781e+02,  3.6386737e+02,  8.9594966e-01],
       [ 3.7379706e+02,  3.6282889e+02,  8.5834831e-01],
       [ 3.5046017e+02,  3.6485599e+02,  9.5850146e-01],
       [ 3.8885910e+02,  3.6172214e+02,  6.5839320e-01],
       [ 3.2807773e+02,  3.7469623e+02,  6.8844420e-01],
       [ 3.3847122e+02,  3.8158102e+02,  3.4103760e-01],
       [ 0.0000000e+00, -3.0000000e+00,  0.0000000e+00]], dtype=float32)

In [7]:
print(len(predictions))
print(dir(predictions[0]))

5
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'add', 'bbox', 'bbox_from_keypoints', 'categories', 'category', 'category_id', 'data', 'decoding_order', 'fixed_bbox', 'fixed_score', 'from_cif_meta', 'frontier_order', 'inverse_transform', 'joint_scales', 'json_data', 'keypoints', 'rescale', 'scale', 'score', 'score_weights', 'set', 'sigmas', 'skeleton', 'skeleton_m1', 'suppress_score_index']


In [8]:
for i in predictions:
    print(i.score)

0.820764676384304
0.8185250992360322
0.7882213203803352
0.6126753506453141
0.6138697357281394


In [9]:
url = "http://121.155.53.218:5000/stream?src=0"
cap = cv2.VideoCapture(url)
prev = 0
FPS = 10

In [10]:
client = MongoClient(host='114.70.235.37', port=27017)
print(client.list_database_names())

['admin', 'config', 'local', 'mydb']


In [11]:
def calcul_xy(xy):
    result = []
    
    neck = [((xy[5][0]+xy[6][0])/2 + xy[0][0]) / 2, ((xy[5][1]+xy[6][1])/2 + xy[0][1]) / 2]
    hip = [(xy[11][0]+xy[12][0])/2, (xy[11][1]+xy[12][1])/2]
    
    result.append((xy[0][0] - neck[0]).item())
    result.append((xy[0][1] - neck[1]).item())
    
    result.append((neck[0] - xy[7][0]).item())
    result.append((neck[1] - xy[7][1]).item())
    
    result.append((neck[0] - xy[8][0]).item())
    result.append((neck[1] - xy[8][1]).item())
    
    result.append((xy[7][0] - xy[9][0]).item())
    result.append((xy[7][1] - xy[9][1]).item())
    
    result.append((xy[8][0] - xy[10][0]).item())
    result.append((xy[8][1] - xy[10][1]).item())
    
    # v5 neck - hip
    result.append((neck[0] - hip[0]).item())
    result.append((neck[1] - hip[1]).item())
    
    result.append((hip[0] - xy[13][0]).item())
    result.append((hip[1] - xy[13][1]).item())
    
    result.append((hip[0] - xy[14][0]).item())
    result.append((hip[1] - xy[14][1]).item())
    
    result.append((xy[13][0] - xy[15][0]).item())
    result.append((xy[13][1] - xy[15][1]).item())
    
    result.append((xy[14][0] - xy[16][0]).item())
    result.append((xy[14][1] - xy[16][1]).item())
    
    return result    

In [12]:

db = client['mydb']
db = db.xy

def input_db(db, data):
    input_data = dict()
    
    input_data['date'] = datetime.now()
    
    for i in range(10):
        
        for j in range(2):
            index = "v" + str(i)
            if j%2 == 0:
                index += 'x'
            else:
                index += 'y'
            
            input_data[index] = data[i*2 + j]
    
    #print(type(input_data['v0x']))
    db.insert_one(input_data)
            

In [13]:
if cap.isOpened(): 
    ret, img = cap.read()
    cur = time.time() - prev
    prepre = None
    
    cnt = 0
    
    while ret:
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #print(type(img))
        img = Image.fromarray(img)
        
        predictions, gt_anns, image_meta = predictor.pil_image(img)
        
        if len(predictions) < 1:
            continue
            
        xy = predictions[0].data
        
        result = calcul_xy(xy)
        input_db(db, result)
        
        ret, img = cap.read()
        cur = time.time() - prev
        cnt+=1
        
        if cnt%100 == 0:
            print(cnt)
else:
    print("url is not opened!!")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


TypeError: a bytes-like object is required, not 'Image'

/bin/bash: conda: command not found
